In [1]:
import sys
import os
from autodp.calibrator_zoo import eps_delta_calibrator,generalized_eps_delta_calibrator, ana_gaussian_calibrator
from autodp import rdp_bank
from autodp.mechanism_zoo import ExactGaussianMechanism, PureDP_Mechanism,SubsampleGaussianMechanism, GaussianMechanism, ComposedGaussianMechanism, LaplaceMechanism
from autodp.transformer_zoo import Composition, AmplificationBySampling
%matplotlib inline 



## Example 1: calibrating noise to privacy budgets.
### Single parameter, composition is not applicable

In [2]:
calibrate = eps_delta_calibrator()
ana_calibrate = ana_gaussian_calibrator()
eps = 0.1
delta = 1e-6

mech1 = calibrate(ExactGaussianMechanism,eps,delta,[0,100],name='GM')
mech2 = ana_calibrate(ExactGaussianMechanism, eps, delta, name='Ana_GM')
print(mech1.name, mech1.params, mech1.get_approxDP(delta))
print(mech2.name, mech2.params, mech2.get_approxDP(delta))

# privacy budget is (0.1, 1e-6), calibrate noise sigma
# mech1 is for Gaussian mechanism and mech2 is for analytical Gaussian mechanism


GM {'sigma': 36.30468875626822} 0.10000000492561108
Ana_GM {'sigma': 36.304691899114694} 0.09999999565548348


## Example 2: Calibration with Gaussian mechanism under composition

In [3]:
#We now have multiple parameters --- params['coeff'] and params['sigma'].


coeff = 20
general_calibrate = generalized_eps_delta_calibrator()
params = {}
params['sigma'] = None
params['coeff'] = 20

mech3 = general_calibrate(ComposedGaussianMechanism, eps, delta, [0,1000],params=params,para_name='sigma', name='Composed_Gaussian')
print(mech3.name, mech3.params, mech3.get_approxDP(delta))
#coeff is the number of composition. The calibrator calibrates the noise with other parameters (coeff) fixed.
#[0,1000] is the range of sigma.


Composed_Gaussian {'sigma': 176.48801940301632, 'coeff': 20} 0.09999999894380397


/usr/local/lib/python3.8/site-packages/scipy/optimize/_optimize.py:2884: RuntimeWarning: invalid value encountered in scalar divide
  w = xb - ((xb - xc) * tmp2 - (xb - xa) * tmp1) / denom


## Example 3:  calibration with SubsampledGaussian mechanism

In [4]:
# We now have three parameters --- params['coeff'], params['prob'] and params['sigma'].
# The coeff and prob are fixed and the calibrator optimize over sigma. We use para_name to denote the parameter that we want to optimize over.

params['prob'] = 0.01
mech4 = general_calibrate(SubsampleGaussianMechanism, eps, delta, [0,1000],params=params,para_name='sigma', name='Subsampled_Gaussian')
print(mech4.name, mech4.params, mech4.get_approxDP(delta))


Subsampled_Gaussian {'prob': 0.01, 'sigma': 3.205947497273216, 'coeff': 20} 0.09999976859421401


## Example 4: Calibration with single parameter for Laplace mechanism

In [5]:
calibrate = generalized_eps_delta_calibrator()

eps = 0.1
delta = 1e-6
mech = calibrate(LaplaceMechanism,eps,delta,[0,100],name='Laplace')
print(mech.name, mech.params, mech.get_approxDP(delta))
#[0, 100] is the range of laplace parameter b.
# to calibrate the noise for the composed Laplace mechanism, we can define a new composed Lapalce mechanism in mechanism.zoo (similar with Example 2)

Laplace {'b': 9.99979887663798} 0.10000001127407157


## Example 5: Calibration of a Complex Mechanism Created using AutoDP API

Remember Example 1 of the tutorial for the new API? This is when we compose Gaussian mechanism for 3 times,  another Gaussian mechanism for 5 times, then a Sparse Vector emchanism (as a pure-DP mechanism) for just once?

The straightforward implementation is the following:


------------
```
from autodp.mechanism_zoo import ExactGaussianMechanism, PureDP_Mechanism
from autodp.transformer_zoo import Composition

sigma1 = 5.0
sigma2 = 8.0

gm1 = ExactGaussianMechanism(sigma1,name='GM1')
gm2 = ExactGaussianMechanism(sigma2,name='GM2')
SVT = PureDP_Mechanism(eps=0.1,name='SVT')

composed_mech = compose([gm1, gm2, SVT], [3, 5, 1])
```
------------

As we can see it was quite easy to create this mechanism object on-the-fly by the mechanism API. In particular, whenever transformers are used, then we can easily spin-up a *Mechanism object* in the runtime, but it doesn't give us a *Mechanissm class definition* that is needed to use our generic calibration tool.  This example illustrates how to convert this into a mechanism class and use our calibator.



In [6]:
# The first step is to package this in a function where the input are the parameters,
# and the output is the mechanism object

def create_complex_mech(sigma1,sigma2,eps, coeffs):
    gm1 = ExactGaussianMechanism(sigma1, name='GM1')
    gm2 = ExactGaussianMechanism(sigma2, name='GM2')
    SVT = PureDP_Mechanism(eps=eps, name='SVT')

    # run gm1 for 3 rounds
    # run gm2 for 5 times
    # run SVT for once
    # compose them with the transformation: compose.
    compose = Composition()
    composed_mech = compose([gm1, gm2, SVT], coeffs)
    return composed_mech

# next we can create it as a mechanism class, which requires us to inherit the base mechanism class,
#  which we import now

from autodp.autodp_core import Mechanism

class Complex_Mechanism(Mechanism):
    def __init__(self, params, name="A_Good_Name"):
        self.name = name
        self.params = params
        mech = create_complex_mech(params['sigma1'], params['sigma2'], params['eps'], params['coeffs'])
        # The following will set the function representation of the complex mechanism
        # to be the same as that of the mech
        self.set_all_representation(mech)

        

        


# Now one can calibrate the mechanism to achieve a pre-defined privacy budget

# Let's say we want to fix sigma1 and sigma2 while tuning the eps parameter from SVT

sigma1 = 5.0
sigma2 = 8.0

coeffs = [3, 5, 1]

# Let's say we are to calibrate the noise to the following privacy budget
eps_budget = 2.5

delta_budget= 1e-6

# declare a general_calibrate "Calibrator"

general_calibrate = generalized_eps_delta_calibrator()

# Set those parameters that we want to fix and leave 
params = {}
params['sigma1'] = sigma1
params['sigma2'] = sigma2
params['coeffs'] = coeffs
params['eps'] = None

# Next, jussts call `general_calibrate` to find the right value of `eps` 
# to give us the mechanism that achieves the required privacy-budget
mech_bob = general_calibrate(Complex_Mechanism, eps_budget, delta_budget, [0,50],params=params,
                             para_name='eps',
                             name='Complex_Mech_Bob')

print(mech_bob.name, mech_bob.params, mech_bob.get_approxDP(delta))


# Notice that so far we support only calibration with one-dimensional parameter while keeping all other parameters fixed.
# Also the range [0,50] needs to be specified by the user so you many need to try a few times to get a feasible solution

# The support for calibrating multiple parameters will be added later

Complex_Mech_Bob {'sigma1': 5.0, 'sigma2': 8.0, 'coeffs': [3, 5, 1], 'eps': 0.4168098107957052} 2.4999997197775063
